In [6]:
import pandas as pd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import numpy as np
project_base = '/Users/icaboalo/Documents/CMU/Projects/FirstProject/'

In [7]:
orders = pd.read_csv(project_base + 'out/orders-clean.csv', chunksize=500000)
cdmx = pd.read_csv(project_base + 'cdmx_polygon.csv')
polanco = pd.read_csv(project_base + 'polanco_polygon.csv')

In [8]:
cdmx_polygon = Polygon(np.column_stack((cdmx.Latitude, cdmx.Longitude)))
polanco_polygon = Polygon(np.column_stack((polanco.Latitude, polanco.Longitude)))

In [9]:
cdmx_orders = []
polanco_orders = []
def filter_cdmx(order):
    try:
        point = Point(order.get('address_lat'), order.get('address_lng'))
        if cdmx_polygon.contains(point) or cdmx_polygon.within(point):
            cdmx_orders.append(order)
            filter_polanco(order)
    except AttributeError:
        print('missing data')
    except:
        print('Error ocurred')
    return order
def filter_polanco(order):
    try:
        point = Point(order.get('address_lat'), order.get('address_lng'))
        if polanco_polygon.contains(point) or polanco_polygon.within(point):
            polanco_orders.append(order)
    except AttributeError:
        print('missing data')
    except:
        print('Error ocurred')
    return order

In [10]:
#orders.apply(lambda s: print(s.get('address_lat', 0)))
for order_batch in orders:
    order_batch.apply(filter_cdmx, axis=1)
    print('cdmx', len(cdmx_orders))
    print('polanco', len(polanco_orders))

cdmx 170109
polanco 14527


In [11]:
cdmx_df = pd.DataFrame(cdmx_orders[1:], columns=['id', 'created_at', 'user_id', 'state', 'payment_method', 'total_value', 'tip', 'user_gender', 'user_birth_date', 'user_vip', 'address', 'address_lat', 'address_lng', 'address_tag', 'city'])
with open('out/cdmx-clean.csv', 'w') as file:  
    file.write(cdmx_df.to_csv(index=False))

In [12]:
polanco_df = pd.DataFrame(polanco_orders, columns=['id', 'created_at', 'user_id', 'state', 'payment_method', 'total_value', 'tip', 'user_gender', 'user_birth_date', 'user_vip', 'address', 'address_lat', 'address_lng', 'address_tag', 'city'])
with open('out/polanco-clean.csv', 'w') as file:
    file.write(polanco_df.to_csv(index=False))